# Reconnaissance faciale via $k$ plus proches voisins ($k$-NN)

>## Partie 2: Reconnaissance faciale en temps réel

## Librairies

In [ ]:
import pickle
import numpy as np
import os

import cv2

from sklearn.neighbors import KNeighborsClassifier


from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

## Algorithme

In [ ]:
# ================== #
# Variables globales #
# ================== #

# Création d'un répertoire data
if not os.path.exists("data"):
    os.mkdir("data")

nb_images_collectees = 10 # Nombre d'image qu'on veut collecter
donnees_visage = [] # Liste vide pour stocker les données des visages

# =============================== #
# Capture des data: nom et images #
# =============================== #

# Définition de la fonction pour capturer une photo
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button'); // bouton pour capture d'image
      capture.textContent = 'Capture';
      div.appendChild(capture);
      // permet l'affichage du flux vidéo de notre caméra en temps réel
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // l'exécution du script s'interromp jusqu'à ce qu'un clic pour la capture soit effectué
      await new Promise((resolve) => capture.onclick = resolve);
      //  capture de l'image, arrêt de la caméra, mise en format jpeg de l'image
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  # On exécute le script JavaScript et on réceptionne l'image capturée sous forme de chaîne de données base64
  data = eval_js('takePhoto({})'.format(quality))
  # on décode la chaîne base64 pour obtenir les données de l'image en binaire
  binary = b64decode(data.split(',')[1])
  # Sauvegarde des données binaires dans un fichier image
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# on charge le modèl qui permet de détecter les visage
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Capture et traitement des images
nom = input("Entrez votre nom: ")  # pour saisir le nom
for i in range(nb_images_collectees):
    filename = take_photo()  # Capture une photo
    image = cv2.imread(filename)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

    for (x, y, w, h) in coordonnees_visage:
        visage = image[y:y+h, x:x+w]
        visage_redimensionne = cv2.resize(visage, (50, 50))
        donnees_visage.append(visage_redimensionne)
        break  # on sauvegarde que le 1er visage détecté

# Conversion en un tableau numpy
donnees_visage = np.array(donnees_visage)

# Enregistrement des noms
noms_path = "data/noms.pkl"
if os.path.exists(noms_path):
    with open(noms_path, 'rb') as file:
        noms = pickle.load(file)
else:
    noms = []

noms += [nom] * nb_images_collectees
with open(noms_path, 'wb') as file:
    pickle.dump(noms, file)

# Enregistrement des visages
visages_path = "data/visages.pkl"
if os.path.exists(visages_path):
    with open(visages_path, 'rb') as file:
        visages_existants = pickle.load(file)
    visages = np.concatenate((visages_existants, donnees_visage), axis=0)
else:
    visages = donnees_visage

with open(visages_path, 'wb') as file:
    pickle.dump(visages, file)


Entrez votre nom: khaoula


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

Shape of visages matrix -->  (30, 50, 50, 3)


In [ ]:
# Attention, pour l'algorithme k-NN, les images doivent être applaties (flattened) en dimension 1
N = len(noms)

visages = visages.reshape(N, -1)

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(visages, noms)


KNeighborsClassifier(n_neighbors=4)

## Exécution

In [ ]:
filename = take_photo()  # Capture d'une photo
image = cv2.imread(filename)
gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

visage_predire = None

for (x, y, w, h) in coordonnees_visage:
    visage = image[y:y+h, x:x+w]
    visage_redimensionne = cv2.resize(visage, (50, 50))
    visage_predire = visage_redimensionne.flatten().reshape(1, -1)  # Aplatir le visage
    break  # Utilise le 1er visage détecté

# Prédiction
if visage_predire is not None:
    prediction = knn.predict(visage_predire)
    print(f"Visage reconnu comme : {prediction[0]}")
else:
    print("Aucun visage détecté pour la prédiction.")


<IPython.core.display.Javascript object>

Visage reconnu comme : khaoula
